# BCC Iron



In [1]:
from pymatgen.core import Structure
from ase.io import read
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np
from weas_widget import WeasWidget


scA = read('Supercell_A.cif')
scA.translate([2.,2.,2.])
scA.wrap()

viewer = WeasWidget()
viewer.from_ase(scA)
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=2.)  # TODO: change me
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7f9a53661fd0>,))

In [2]:
scB = read('Supercell_B.cif')
scB.translate([2.,2.,2.])
scB.wrap()

viewer = WeasWidget()
viewer.from_ase(scB)
viewer.avr.model_style = 1
viewer.avr.bond.add_bond_pair('Fe', 'H', max=2.5)  # TODO: change me
viewer

WeasWidget(children=(<weas_widget.base_widget.BaseWidget object at 0x7f9a537db390>,))

## Let's compute harmonic zero point motion energy

For the sake of simpplicity, we only consider a simple harmonic approximation.

In [3]:
# Potential for tetrahedral site. Displacements are in Angstrom, energies in eV

# Potential for octahedral site. Displacements are in Angstrom, energies in eV

In [4]:
# Can you compute the zero point energy?

# Local fields and distortions

In [9]:
import numpy as np
from ase.io import read
from muesr.core.sample import Sample                   # Retains all the sample info.
from muesr.utilities import mago_add, show_structure   # For magnetic structure description and visualization
from muesr.utilities import muon_find_equiv            # For finding and including the symmetry equivalent muon positions in the calculation

from muesr.engines.clfc import locfield, find_largest_sphere # Does the sum and returns the local field in its various contributions

np.set_printoptions(suppress=True,precision=5)



In [67]:
fe=Sample()
load_cif(fe, 'Fe.cif')
print_cell(fe)

	 Lattice vectors:
	   a    2.866500000000000    0.000000000000000    0.000000000000000
	   b    0.000000000000000    2.866500000000000    0.000000000000000
	   c    0.000000000000000    0.000000000000000    2.866500000000000
	 Atomic positions (fractional):
	     1 Fe  0.00000000000000  0.00000000000000  0.00000000000000  55.845
	     2 Fe  0.50000000000000  0.50000000000000  0.50000000000000  55.845


True

In [68]:
fe

Sample status: 

 Crystal structure:           Yes
 Magnetic structure:          No
 Muon position(s):            No
 Symmetry data:               Yes

In [69]:
fe.sym


Spacegroup(229, setting=1)

In [70]:
fe.add_muon([0.50,0.25,0.0]);

In [71]:
muon_find_equiv(fe);

In [72]:
fe

Sample status: 

 Crystal structure:           Yes
 Magnetic structure:          No
 Muon position(s):            12 site(s)
 Symmetry data:               Yes

In [73]:
fe.new_mm()
fe.mm.k=np.array([0.0,0.0,0.0])
fe.mm.fc= np.array([[0.0+0.j, 0.0+0.j, 2.22+0.j],
                     [0.0+0.j, 0.0+0.j, 2.22+0.j]])

In [80]:
radius=find_largest_sphere(fe,[100, 100, 100])
r=locfield(fe, 's', [100, 100, 100] ,radius)

B_dip=np.zeros([len(fe.muons),3])
B_Lor=np.zeros([len(fe.muons),3])
B_Cont=np.zeros([len(fe.muons),3])
B_Tot=np.zeros([len(fe.muons),3])

for i in range(len(fe.muons)):
    B_dip[i]=r[i].D
    B_Lor[i]=r[i].L
    r[i].ACont = -0.0644
    B_Cont[i]=r[i].C
    B_Tot[i]=r[i].T

In [81]:
print("Dipolar Field for all the 12 tetrahedral equivalent sites")
print(B_dip)

# This is and should be same for all the equivalent sites
print("The Lorentz field is {:4.3f} {:4.3f} {:4.3f}".format(*tuple(B_Lor[0])))

print("The contact field is {:4.3f} T".format(np.linalg.norm(B_Cont[0])))

print("Dipolar average of 1 parallel site and 2 antiparallel sites is {:4.5f} T".format(np.linalg.norm(np.sum(B_dip,axis=0))))

print("The sum of the Contact and the Lorentz field is {:4.5f} T".format(np.linalg.norm(B_Lor[0] + B_Cont[0])))

Dipolar Field for all the 12 tetrahedral equivalent sites
[[ 0.     -0.      0.2654]
 [ 0.      0.      0.2654]
 [ 0.     -0.     -0.5308]
 [ 0.     -0.     -0.5308]
 [ 0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.     -0.      0.2654]
 [-0.      0.     -0.5308]
 [-0.     -0.     -0.5308]
 [-0.      0.      0.2654]
 [-0.      0.      0.2654]]
The Lorentz field is 0.000 0.000 0.732
The contact field is 1.111 T
Dipolar average of 1 parallel site and 2 antiparallel sites is 0.00000 T
The sum of the Contact and the Lorentz field is 0.37852 T
